In [2]:
import cv2

In [3]:
img_ok = cv2.imread('../team/images/true_ok/GSY827AN7A1356_AAO11960K_PKT10_CM1EQSUA0012_20220711210457_DirectLight_OK.jpg')
img_ng = cv2.imread('../team/images/true_ng/GSY827AN7B0519_AAO12705K_PKT08_CM1EQSUA0011_20220711213213_DirectLight_NG.jpg')
img_ng2 = cv2.imread('../team/images/true_ng/GSY827AN7A1641_AAO19043K_PKT13_CM1EQSUA0012_20220711220653_DirectLight_NG.jpg')
img_over = cv2.imread('../team/images/overkill/GSY827AN7B0355_AAO04537K_PKT01_CM1EQSUA0011_20220711234650_DirectLight_NG.jpg')


In [4]:
def img_resize(img, resize_size = 1600):
    width_origin = img.shape[1]
    height_origin = img.shape[0]
    hw_list = [height_origin, width_origin]

    img_ratio = float(resize_size)/ max(hw_list)
    # print(img_ratio)
    max_index = hw_list.index(max(hw_list))
    if max_index ==0:
        img = cv2.resize(img, (int(width_origin * img_ratio), resize_size),interpolation=cv2.INTER_CUBIC)
    else:
        img = cv2.resize(img, (resize_size,int(height_origin * img_ratio)),interpolation=cv2.INTER_CUBIC)

    return img


In [5]:
def find_large_label(img, img_bin, show_img = False):
    test_img = img.copy()
    _, labels, stats, centroids = cv2.connectedComponentsWithStats(img_bin,)
    stats = sorted(stats, key = lambda x: x[4],reverse=True)
    if show_img:
        for i, rec in enumerate(stats[:2]):
            x,y,w,h, area = rec
            cv2.rectangle(test_img, (x,y,w,h), (0,0,255),thickness=8)

        cv2.imshow('items', img_resize(test_img, 600))
        cv2.imshow('items_bin', img_resize(img_bin, 600))
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        # print(stats[0])
    if stats[0][1]==0:
        return stats[1]
    return stats[0]

In [6]:
def get_threshold(img,gray,bin_inverse = True):
    if bin_inverse:
        _, img_bin = cv2.threshold(gray, -1,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    else:
        _, img_bin = cv2.threshold(gray, -1,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    return img_bin


In [7]:
def preprocess(img, bin_inverse = True, only_bin = False):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_bin = get_threshold(img,gray,bin_inverse = True)

    large_stat = find_large_label(img, img_bin)
    # print(large_stat)
    try:
        item_img = img[large_stat[1]:large_stat[1]+large_stat[3],large_stat[0]:large_stat[0]+large_stat[2], :].copy()
        item_gray = gray[large_stat[1]:large_stat[1]+large_stat[3],large_stat[0]:large_stat[0]+large_stat[2]].copy()
    except:
        item_img, item_gray= img,gray

    item_bin = get_threshold(item_img,item_gray,bin_inverse = False)
    return  item_img, item_gray, item_bin

In [8]:
# img_ng = cv2.imread('../team/images/true_ng/GSY827AN7D2341_AAO08637K_PKT15_CM1EQSUA0012_20220714014226_DirectLight_NG.jpg')
# gray = cv2.cvtColor(img_ng, cv2.COLOR_BGR2GRAY)
# img_bin = get_threshold(img_ng,gray,bin_inverse = True)
# find_large_label(img_ng, img_bin, show_img = True)
# item_img, item_gray, item_bin = preprocess(img_ng)

# cv2.imshow('img', img_resize(img_ng, 600))
# cv2.imshow('items', img_resize(item_img, 600))
# cv2.waitKey(0)
# cv2.destroyAllWindows()



In [9]:
import numpy as np
def img_mask(img):
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(img_hsv,(0,0,0),(0,0,100))
    imask = mask>0
    col = np.zeros_like(img_hsv, np.uint8)
    col[imask] = img_hsv[imask]
    col = cv2.cvtColor(col,cv2.COLOR_HSV2BGR)
    return col

In [10]:
import math
# 두 점의 각도 계산
def get_angle(pt1, pt2):
    xd = abs(pt2[0] - pt1[0])
    yd = abs(pt2[1] - pt1[1])
    if xd == 0:
        radian = 0.5*math.pi
    else:
        radian = math.atan(yd/xd)
    return radian


In [11]:
get_angle([0,0], [1,-(3**0.5)])*180/math.pi

59.99999999999999

In [12]:
# 각도를 계산할 두 꼭짓점 찾기
def get_points(cnt):
    rect = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(rect)   # 중심점과 각도를 4개의 꼭지점 좌표로 변환
    box = np.int0(box)          # 정수로 변환
    # pt1_ind = cnt[:,:,1].argmin()
    # pt2_ind = cnt[:,:,0].argmin()

    return box

In [13]:
math.sin(1/6*math.pi)

0.49999999999999994

In [14]:
def cnt_test(cnt):
    volum_ratio_bound = 0.003
    """_summary_

    Args:
        cnt (_type_): _description_

    Returns:
        _type_: _description_
    """
    area = cv2.contourArea(cnt)
    arcLen = cv2.arcLength(cnt, closed=True)
    box = get_points(cnt)
    s_box = sorted(box, key = lambda x: x[0]+x[1])
    # print(s_box)
    point1, point2 = s_box[0],s_box[2]
    angle = get_angle(point1, point2)
    # print(max_area, area,round(area/max_area,3), arcLen)
    sin, cos = math.sin(angle), math.cos(angle)
    cal_max_box = ((arcLen**2)/4 - 2*area)*(sin*cos)+area
    # max_box = (w_max-w_min)*(h_max-h_min)
    max_box = cv2.contourArea(box)
    print(round(cal_max_box/max_box,3), angle*180/math.pi)
    # print(box[1:3], angle*180/math.pi)
    if  cal_max_box/max_box < 1-volum_ratio_bound+0.001 or cal_max_box/max_box > 1+volum_ratio_bound-0.001 :
        pred = 'NG'
    else:
        pred = 'ok'
    return point1, point2, pred

In [15]:
def cnt_test2(cnt):
    volum_ratio_bound = 0.004
    """_summary_

    Args:
        cnt (_type_): _description_

    Returns:
        _type_: _description_
    """
    area = cv2.contourArea(cnt) 
    rect = cv2.minAreaRect(cnt) 
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    max_area = cv2.contourArea(box) 
    print(max_area, area,round(area/max_area,3))

    if  area/max_area < 1-volum_ratio_bound +0.001 or area/max_area > 1+volum_ratio_bound -0.001:
        pred = 'NG'
    else:
        pred = 'OK'
    return box, pred

In [25]:
def check_contain(img_shape, cnt):
    center_bound = 100
    (x,y), radius = cv2.minEnclosingCircle(cnt)
    # center = (int(x), int(y))
    radius = int(radius)
    center_diff = ((int(img_shape[0]/2)-int(y))**2+(int(img_shape[1]/2) -int(x))**2)**0.5
    # print(center, (img_shape[1]/2,img_shape[0]/2), center_diff)
    if center_diff < center_bound:
        return True
    return False

In [105]:
def carrierTest(item_img, box):
    box = np.array(sorted(box, key = lambda x: sum(x)))
    carrier_range = 80
    epoxy_range = carrier_range-20
    
    carrierBox = box.copy()
    epoxyBox = box.copy()
    carrierBox[0] = np.array([box[1,0]-carrier_range, box[1,1]+carrier_range+7])
    carrierBox[1] = np.array([box[0,0]-carrier_range, box[0,1]-carrier_range-7])
    carrierBox[2] = np.array([box[2,0]+carrier_range, box[2,1]-carrier_range-7])
    carrierBox[3] = np.array([box[3,0]+carrier_range, box[3,1]+carrier_range+7])
    epoxyBox[0] = np.array([box[1,0]-epoxy_range, box[1,1]+epoxy_range+7])
    epoxyBox[1] = np.array([box[0,0]-epoxy_range, box[0,1]-epoxy_range-7])
    epoxyBox[2] = np.array([box[2,0]+epoxy_range, box[2,1]-epoxy_range-7])
    epoxyBox[3] = np.array([box[3,0]+epoxy_range, box[3,1]+epoxy_range+7])
    print(carrierBox)
    carrier_img = item_img[carrierBox[1,1]:carrierBox[3,1],carrierBox[0,0]:carrierBox[3,0], :].copy()
    cv2.imshow('carrier_img', img_resize(carrier_img, 600))
    cv2.waitKey(0)
    # cv2.destroyAllWindows()

    pred = 'NG'

    return pred, carrierBox, epoxyBox

In [104]:
def find_contours(img, ip = None):
    # img = cv2.resize(img, (0,0), fx = 0.3, fy = 0.3, interpolation=cv2.INTER_CUBIC)
    item_img, item_gray, item_bin = preprocess(img)
    col = img_mask(item_img)
    # contour, hierachy = cv2.findContours(item_bin, cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    contour, hierachy = cv2.findContours(item_bin, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    result = False
    for i, cnt in enumerate(contour):

        if cv2.contourArea(cnt) > 1500000:
            if check_contain(item_img.shape, cnt):
                cv2.drawContours(item_img, [cnt], 0,(255,0,0),5)
                result = True
                break
        

    if ip:
        cv2.putText(img, ip, (10,60),cv2.FONT_HERSHEY_SIMPLEX,3,(0,255,0),3)
    
    pred = 'NG'
    if result:
        box, pred = cnt_test2(cnt)
        cv2.drawContours(item_img, [box], 0,(0,255,0),5)

        # print(box)
        # j=1
        # cols = [(255,255,255),(0,0,255),(0,255,0),(255,0,0)]
        # point1x,point1y = point1
        # point2x,point2y = point2
        
        # cv2.circle(item_img,point1,5,cols[1],10) 
        # cv2.circle(item_img,point2,5,cols[2],10) 

    # carrier epoxy test
    if pred == 'OK':
        pred, carrierBox, epoxyBox = carrierTest(item_img, box)
        cv2.drawContours(item_img, [carrierBox], 0,(0,0,255),3)
        cv2.drawContours(item_img, [epoxyBox], 0,(40,128,128),3)

    cv2.putText(item_img, 'predicted '+pred, (10,60),cv2.FONT_HERSHEY_SIMPLEX,3,(0,255,0),3)


    # print(item_img.shape)
    # cv2.imshow('img', img)
    cv2.imshow('item_img', img_resize(item_img,600))
    # cv2.imshow('col', col)
    # cv2.imshow('bin', item_bin)
    
    key_val = cv2.waitKey(0)
    cv2.destroyAllWindows()

    return key_val



In [29]:
test_img = cv2.imread('O:/lg/team/images/true_ng/GSY827BN8E0035_AAO19345K_PKT01_CM1EQSUA0012_20220814060338_DirectLight_NG.jpg')


In [64]:
find_contours(test_img, ip = 'NG')

2135625.0 2101757.5 0.984


13

In [106]:
# find_contours(img_ng, ip = 'NG')
# find_contours(img_ng2, ip = 'NG')
find_contours(img_ok, ip = 'ok')
# find_contours(img_over, ip = 'ok')

2105060.0 2100386.0 0.998
[[ 306 1611]
 [ 308  181]
 [2144  183]
 [2142 1613]]


13

In [ ]:
import os
import random
import time
def check_img(kind = 'overkill'):
    random.seed(time.time_ns()%10000)
    if kind == 'all':
        paths = os.listdir('../team/images/')
        img_paths = []
        for p in paths:
            file_path = '../team/images/'+p+'/'
            img_paths = list(map(lambda x: [file_path+ x, p], os.listdir(file_path)))
        while True:
            ip = random.choice(img_paths)
            img = cv2.imread(ip[0])
            
            key_val= find_contours(img, ip =ip[1])
            
            if key_val == 113:
                break

    else:
        file_path = '../team/images/'+kind+'/'
        img_paths = os.listdir(file_path)
        for i in range(len(img_paths)):
            img = cv2.imread(file_path+img_paths[i])
            key_val = find_contours(img,  ip = kind)
            if key_val == 113: # ord('q')
                break

In [ ]:
check_img('true_ng')

2205658.0 2104501.0 0.954
2235969.5 2108099.5 0.943
1801822.5 1747312.0 0.97
1808898.0 1753930.0 0.97
1800366.0 1711622.5 0.951
1804656.0 1715527.5 0.951
2104431.0 2100317.5 0.998
2191266.0 2103335.5 0.96
2236056.0 2109826.5 0.944
2076676.0 2066804.5 0.995
2173970.0 2102896.5 0.967
2143608.0 2100875.5 0.98
2175452.0 2105838.5 0.968
2186288.0 2103713.5 0.962
2132708.0 2100826.0 0.985
2153920.0 2081274.0 0.966
2189210.0 2076559.5 0.949
2081676.0 2067448.5 0.993
2083436.0 2069483.5 0.993
2111628.0 2075338.0 0.983
2082915.0 2068040.0 0.993
2178330.0 2080873.5 0.955
2189799.0 2076976.5 0.948
2194071.0 2077321.0 0.947
2134007.0 2102391.0 0.985
2152004.0 2118561.5 0.984
2180193.0 2080852.0 0.954
2119535.0 2074220.5 0.979
2103619.5 2099770.5 0.998
2204881.5 2114251.0 0.959
2184372.0 2094920.0 0.959
2128389.5 2096508.0 0.985
2156443.5 2109963.0 0.978
2176842.0 2102138.5 0.966
2185372.0 2108174.0 0.965
2209909.5 2107103.5 0.953
2238900.0 2107830.0 0.941
2308529.0 2109405.0 0.914


In [ ]:
check_img('overkill')

2105899.0 2100728.0 0.998
2089587.0 2073703.5 0.992
2084182.0 2072606.5 0.994
2103908.0 2098402.5 0.997
2104472.0 2099911.0 0.998
2103800.0 2100086.5 0.998
2075845.0 2071898.5 0.998
2105062.0 2101370.0 0.998
2105916.0 2101372.5 0.998
2105060.0 2099527.5 0.997
2105144.0 2099922.5 0.998
2104431.0 2100340.5 0.998
2102125.0 2100229.0 0.999


In [ ]:
check_img('true_ok')

2105060.0 2100386.0 0.998
2105895.5 2100694.0 0.998
2102125.0 2100589.5 0.999
2104435.5 2100415.0 0.998
2103848.0 2100806.0 0.999
2103800.0 2100703.0 0.999
2103800.0 2101530.5 0.999
2102125.0 2100919.5 0.999
2100870.0 2100087.5 1.0
2105698.0 2100942.0 0.998
2105895.5 2100973.5 0.998
2105076.0 2100134.0 0.998
2105068.0 2101300.0 0.998
2105058.0 2101342.5 0.998
2105176.0 2100950.0 0.998
